In [34]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f 

In [7]:
spark = SparkSession.builder.appName("Sandbox").getOrCreate()

In [20]:
df = spark.read.csv("zipcodes.csv", header=True, inferSchema=True)
df.show(3)

+------------+-------+-----------+-------------------+-----+--------------+-----+------+-----+-----+-----+-----------+-------+--------------------+--------------------+-------------+---------------+-------------------+----------+-----+
|RecordNumber|Zipcode|ZipCodeType|               City|State|  LocationType|  Lat|  Long|Xaxis|Yaxis|Zaxis|WorldRegion|Country|        LocationText|            Location|Decommisioned|TaxReturnsFiled|EstimatedPopulation|TotalWages|Notes|
+------------+-------+-----------+-------------------+-----+--------------+-----+------+-----+-----+-----+-----------+-------+--------------------+--------------------+-------------+---------------+-------------------+----------+-----+
|           1|    704|   STANDARD|        PARC PARQUE|   PR|NOT ACCEPTABLE|17.96|-66.22| 0.38|-0.87|  0.3|         NA|     US|     Parc Parque, PR|NA-US-PR-PARC PARQUE|        false|           null|               null|      null| null|
|           2|    704|   STANDARD|PASEO COSTA DEL SUR|  

In [21]:
df.printSchema()

root
 |-- RecordNumber: integer (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- LocationType: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Xaxis: double (nullable = true)
 |-- Yaxis: double (nullable = true)
 |-- Zaxis: double (nullable = true)
 |-- WorldRegion: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LocationText: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Decommisioned: boolean (nullable = true)
 |-- TaxReturnsFiled: integer (nullable = true)
 |-- EstimatedPopulation: integer (nullable = true)
 |-- TotalWages: integer (nullable = true)
 |-- Notes: string (nullable = true)



In [22]:
df.head()

Row(RecordNumber=1, Zipcode=704, ZipCodeType='STANDARD', City='PARC PARQUE', State='PR', LocationType='NOT ACCEPTABLE', Lat=17.96, Long=-66.22, Xaxis=0.38, Yaxis=-0.87, Zaxis=0.3, WorldRegion='NA', Country='US', LocationText='Parc Parque, PR', Location='NA-US-PR-PARC PARQUE', Decommisioned=False, TaxReturnsFiled=None, EstimatedPopulation=None, TotalWages=None, Notes=None)

In [36]:
df.groupBy('State').count().sort(f.desc('count')).show(10)

+-----+-----+
|State|count|
+-----+-----+
|   PR|    6|
|   FL|    4|
|   AL|    3|
|   NC|    3|
|   TX|    3|
|   AZ|    2|
+-----+-----+



In [ ]:
df.createOrReplaceTempView('zipcodes')
spark.sql('select distinct Country from zipcodes').show()

+-------+
|Country|
+-------+
|     US|
+-------+



In [ ]:
df = df.drop('Notes')

In [ ]:
df.na.drop(subset=['TotalWages']).show()

+------------+-------+-----------+-----------+-----+--------------+-----+-------+-----+-----+-----+-----------+-------+---------------+--------------------+-------------+---------------+-------------------+----------+
|RecordNumber|Zipcode|ZipCodeType|       City|State|  LocationType|  Lat|   Long|Xaxis|Yaxis|Zaxis|WorldRegion|Country|   LocationText|            Location|Decommisioned|TaxReturnsFiled|EstimatedPopulation|TotalWages|
+------------+-------+-----------+-----------+-----+--------------+-----+-------+-----+-----+-----+-----------+-------+---------------+--------------------+-------------+---------------+-------------------+----------+
|       61392|  76177|   STANDARD| FORT WORTH|   TX|       PRIMARY|32.75| -97.33| -0.1|-0.83| 0.54|         NA|     US| Fort Worth, TX| NA-US-TX-FORT WORTH|        false|           2126|               4053| 122396986|
|       61393|  76177|   STANDARD|   FT WORTH|   TX|    ACCEPTABLE|32.75| -97.33| -0.1|-0.83| 0.54|         NA|     US|   Ft Wor

In [102]:
df_population = df.na.drop(subset=['EstimatedPopulation'])
# df_population.show(10)
# df_population.groupBy('State').sum('EstimatedPopulation')["sum(EstimatedPopulation)"]
df_grouped = df_population.groupBy('State').agg(f.sum('EstimatedPopulation').alias('sum_population'),f.sum('TotalWages').alias('sum_wages')).withColumn('Wage/Person', f.col('sum_wages')/f.col('sum_population')).sort(f.desc('Wage/Person'))
df_grouped.show()
# df_population.withColumn('StatePopulation',df_population.groupBy('State').agg(f.sum('EstimatedPopulation').alias('sum_population'))['sum_population'])
# df_population.withColumn('StatePopulation',df_grouped)
# .select(['State','StatePopulation'])

+-----+--------------+----------+------------------+
|State|sum_population| sum_wages|       Wage/Person|
+-----+--------------+----------+------------------+
|   TX|          8106| 244793972| 30199.10831482852|
|   AL|          9054| 190653116|21057.335542301746|
|   AZ|         52329|1034793195| 19774.75577595597|
|   FL|          9633| 169508062| 17596.60147409945|
|   NC|         18710| 274673284|14680.560342063069|
+-----+--------------+----------+------------------+



In [ ]:
df.na.drop(subset=['Zipcode','Lat','Long']).show()

+------------+-------+-----------+-------------------+-----+--------------+-----+-------+-----+-----+-----+-----------+-------+--------------------+--------------------+-------------+---------------+-------------------+----------+
|RecordNumber|Zipcode|ZipCodeType|               City|State|  LocationType|  Lat|   Long|Xaxis|Yaxis|Zaxis|WorldRegion|Country|        LocationText|            Location|Decommisioned|TaxReturnsFiled|EstimatedPopulation|TotalWages|
+------------+-------+-----------+-------------------+-----+--------------+-----+-------+-----+-----+-----+-----------+-------+--------------------+--------------------+-------------+---------------+-------------------+----------+
|           1|    704|   STANDARD|        PARC PARQUE|   PR|NOT ACCEPTABLE|17.96| -66.22| 0.38|-0.87|  0.3|         NA|     US|     Parc Parque, PR|NA-US-PR-PARC PARQUE|        false|           null|               null|      null|
|           2|    704|   STANDARD|PASEO COSTA DEL SUR|   PR|NOT ACCEPTABLE|1

In [103]:
df.filter('Long < -100.0').show()

+------------+-------+-----------+----+-----+------------+-----+-------+-----+-----+-----+-----------+-------+------------+-------------+-------------+---------------+-------------------+----------+
|RecordNumber|Zipcode|ZipCodeType|City|State|LocationType|  Lat|   Long|Xaxis|Yaxis|Zaxis|WorldRegion|Country|LocationText|     Location|Decommisioned|TaxReturnsFiled|EstimatedPopulation|TotalWages|
+------------+-------+-----------+----+-----+------------+-----+-------+-----+-----+-----+-----------+-------+------------+-------------+-------------+---------------+-------------------+----------+
|       39827|  85209|   STANDARD|MESA|   AZ|     PRIMARY|33.37|-111.64| -0.3|-0.77| 0.55|         NA|     US|    Mesa, AZ|NA-US-AZ-MESA|        false|          14962|              26883| 563792730|
|       39828|  85210|   STANDARD|MESA|   AZ|     PRIMARY|33.38|-111.84|-0.31|-0.77| 0.55|         NA|     US|    Mesa, AZ|NA-US-AZ-MESA|        false|          14374|              25446| 471000465|
+----

In [106]:
df.filter( (f.col('Lat') > 40.0) | (f.col('Long') < -100.0) ).show()

+------------+-------+-----------+----+-----+------------+-----+-------+-----+-----+-----+-----------+-------+------------+-------------+-------------+---------------+-------------------+----------+
|RecordNumber|Zipcode|ZipCodeType|City|State|LocationType|  Lat|   Long|Xaxis|Yaxis|Zaxis|WorldRegion|Country|LocationText|     Location|Decommisioned|TaxReturnsFiled|EstimatedPopulation|TotalWages|
+------------+-------+-----------+----+-----+------------+-----+-------+-----+-----+-----+-----------+-------+------------+-------------+-------------+---------------+-------------------+----------+
|       39827|  85209|   STANDARD|MESA|   AZ|     PRIMARY|33.37|-111.64| -0.3|-0.77| 0.55|         NA|     US|    Mesa, AZ|NA-US-AZ-MESA|        false|          14962|              26883| 563792730|
|       39828|  85210|   STANDARD|MESA|   AZ|     PRIMARY|33.38|-111.84|-0.31|-0.77| 0.55|         NA|     US|    Mesa, AZ|NA-US-AZ-MESA|        false|          14374|              25446| 471000465|
+----